In [3]:
import datetime as dt
import glob
import netCDF4 as nc

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import xarray as xr
import gsw
from gsw import *
import cmocean
from datetime import datetime

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config["data_dir"] = './cartopy_shapefiles'

import yaml

os.getcwd()

'/home1/datahome/mlejeune'

In [4]:
with open(r'/home1/datahome/mlejeune/configuration.yaml') as file:
    configuration = yaml.load(file, Loader=yaml.FullLoader)

    print("File version: ", configuration["version"])

File version:  0.1


In [12]:
# Clean OutPut
Delete = False;
haveToCreate = False;

if(Delete):
    haveToCreate = True;
    if len(os.listdir(configuration['CORA_OUTPUT'])) != 0:
    
         arr = os.listdir(configuration['CORA_OUTPUT'])
         for x in arr:
             os.remove((configuration['CORA_OUTPUT'] + x))
             print("delete file: ", (configuration['CORA_OUTPUT'] + x))
             
    else:    
        print("Directory is empty")
    

delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930128_PR_OC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930205_PR_CT_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19931019_PR_IC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930906_PR_IC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930927_PR_IC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930901_PR_IC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930525_PR_OC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930611_PR_IC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19931205_PR_OC_Nordic_Sea.nc
delete file:  /home1/datahome/mlejeune/cora_Nordic_Sea/test/CO_DMQCGL01_19930818_PR_OC_Nord

In [4]:
%%time
haveToCreate = True
if(haveToCreate):
    cora_rep = configuration['CORA_DATA_REP']
    data_rep = configuration['CORA_OUTPUT']

    for yy in range(2019,2020):
        listfile = set(glob.glob(cora_rep + str(yy) + '/CO_*.nc')) - set(glob.glob(cora_rep + str(yy) + '/*MO.nc')) - set(glob.glob(cora_rep + str(yy) + '/*TE.nc')) - set(glob.glob(cora_rep + str(yy) + '/*DB.nc'))
        count = list()
        for myfile in listfile:
            filename = os.path.basename(myfile)
            ds = nc.Dataset(myfile, 'r')  # r+
            dm = ds.variables['DATA_MODE'][:]
            dcref = ds.variables['DC_REFERENCE'][:,:]
            lon   = ds.variables['LONGITUDE'][:]
            lat   = ds.variables['LATITUDE'][:]
            juld  = ds.variables['JULD'][:]
            platform_number = ds.variables['PLATFORM_NUMBER'][:]
            ind     = list()
            mydcref = list()
            mypfnum = list()
            for ii in range(0,len(dm)):
                suffix  =''
                #REGION
                if lon[ii]<=configuration['longitude2_oceans'] and lon[ii]>=configuration['longitude1_oceans'] and lat[ii]<=configuration['latitude1_oceans'] and lat[ii]>=configuration['latitude2_oceans']:
                        
                    #ADJUSTED IF IT EXIST
                    if dm[ii] == b'A' or dm[ii] == b'D':
                        suffix = '_ADJUSTED'
                    #TEMP AND PSAL AND PRES
                    if 'TEMP' in ds.variables.keys() and 'PSAL' in ds.variables.keys() and 'PRES' in ds.variables.keys():
                        ind.append(ii)
                        dc = b''.join(dcref[ii,dcref[ii,:].mask== False])
                        dc = dc.decode('utf-8')
                        mydcref.append(dc)
                        pf = b''.join(platform_number[ii,platform_number[ii,:].mask== False])
                        mypfnum.append(pf)
                        
            if len(ind)>0:
                count.append(len(ind))
                #RETRIEVE QC FOR ind AND CREATE MASK FROM IT
                mask_TEMP = np.isin(ds.variables['TEMP' + suffix  + '_QC'][ind,:].data, b'1')
                mask_PSAL = np.isin(ds.variables['PSAL' + suffix  + '_QC'][ind,:].data, b'1')
                mask_PRES = np.isin(ds.variables['PRES' + suffix  + '_QC'][ind,:].data, b'1')
                #MAKE A SINGLE MASK FROM THE COMBINED THREE OTHERS
                mask = mask_TEMP & mask_PSAL & mask_PRES
                #EXPORT IN DATASET XARRAY
                dsxr = xr.Dataset(
                data_vars=dict(
                    TEMP = (["N_PROF","N_PRES"], ma.masked_array(ds.variables['TEMP' + suffix ][ind,:], mask=~mask)),
                    PSAL = (["N_PROF","N_PRES"], ma.masked_array(ds.variables['PSAL' + suffix ][ind,:], mask=~mask)),
                    PRES = (["N_PROF","N_PRES"], ma.masked_array(ds.variables['PRES' + suffix ][ind,:], mask=~mask)),
                    DC_REFERENCE    = (["N_PROF"], mydcref),
                    PLATFORM_NUMBER = (["N_PROF"], mypfnum)
                ),
                coords=dict(
                    LON  = (["N_PROF"], lon[ind]),
                    LAT  = (["N_PROF"], lat[ind]),
                    JULD = (["N_PROF"],juld[ind])
                ),
                attrs=dict(description="CORA subsampled : Selection of profiles with (1) TEMP, PSAL and PRES present, (2) adjusted data if adjusted exist "),
                )
                dsxr.JULD.attrs["units"] = "days since 1950-01-01"
                dsxr['DC_REFERENCE'] = dsxr['DC_REFERENCE'].astype('|S8')
                dsxr.to_netcdf(data_rep + myfile[50:76] + "_Nordic_Sea.nc")
            ds.close()
        print('Extracted a total of ' + str(sum(count)) + ' TS profiles in the Nordic sea Region, for the year ' + str(yy) + '.')

Extracted a total of 7156 TS profiles in the Nordic sea Region, for the year 2019.
CPU times: user 42 s, sys: 8.6 s, total: 50.6 s
Wall time: 4min 46s


In [5]:
%%time
pi = configuration['PI']

listfile = glob.glob(configuration['CORA_OUTPUT'] + '*.nc');
data_rep = configuration['CORA_OUTPUT'];

total_files = len(listfile)
datasets = []
nb_prof = 0
count = 0

for myfile in listfile:
    ds = xr.open_dataset(myfile)
    ds = ds.dropna('N_PROF', how = "all", subset=['TEMP', 'PSAL'])
    N = len(ds.N_PROF)
    #Exclude empty files
    if N>0:
        TEMPi = np.empty((N,len(pi)))
        TEMPi[:] = np.nan
        PSALi = np.empty((N,len(pi)))
        PSALi[:] = np.nan
        TEMPs = np.empty((N,len(pi)))
        TEMPs[:] = np.nan
        PSALs = np.empty((N,len(pi)))
        PSALs[:] = np.nan
        for n in range(0,N):
            TEMPi[n,:] = np.interp(pi, ds.PRES[n,:], ds.TEMP[n,:],right=np.nan)
            PSALi[n,:] = np.interp(pi, ds.PRES[n,:], ds.PSAL[n,:],right=np.nan)
            TEMPs[n,:] = np.interp(pi, ds.PRES[n,:], ds.TEMP[n,:],left=np.nan,right=np.nan)
            PSALs[n,:] = np.interp(pi, ds.PRES[n,:], ds.PSAL[n,:],left=np.nan,right=np.nan)
            #EXPORT INTERPOLATED PROFILES
        dsi = xr.Dataset(
            data_vars=dict(
                TEMP_SURF = (["N_PROF","PRES_INTERPOLATED"], TEMPs),
                PSAL_SURF = (["N_PROF","PRES_INTERPOLATED"], PSALs),
                TEMP_INTERP = (["N_PROF","PRES_INTERPOLATED"], TEMPi),
                PSAL_INTERP = (["N_PROF","PRES_INTERPOLATED"], PSALi),
                DC_REFERENCE    = (["N_PROF"], ds.DC_REFERENCE.data),
                PLATFORM_NUMBER = (["N_PROF"], ds.PLATFORM_NUMBER.data)),
            coords=dict(
                LON  = (["N_PROF"], ds.LON.data),
                LAT  = (["N_PROF"], ds.LAT.data),
                PRES_INTERPOLATED  = (["PRES_INTERPOLATED"], pi),
                JULD = (["N_PROF"],ds.JULD.data)))
        datasets.append(dsi)
        nb_prof = nb_prof + len(dsi.N_PROF)
    count = count+1
    if count % 1000 == 0:
        print(f"{(count/total_files)*100}% done")
        print(f"{nb_prof} profiles processed")
        
print('finished, concatenation starting')
ds_interp = xr.concat(datasets, dim='N_PROF')
ds_interp.to_netcdf(data_rep + "ds_interp.nc")

3.719269535463235% done
6562 profiles processed
7.43853907092647% done
14219 profiles processed
11.157808606389704% done
24668 profiles processed
14.87707814185294% done
35105 profiles processed
18.596347677316174% done
44190 profiles processed
22.315617212779408% done
51558 profiles processed
26.03488674824265% done
57823 profiles processed
29.75415628370588% done
63282 profiles processed
33.473425819169115% done
68121 profiles processed
37.19269535463235% done
73856 profiles processed
40.91196489009559% done
79790 profiles processed
44.631234425558816% done
84986 profiles processed
48.350503961022056% done
90749 profiles processed
52.0697734964853% done
98078 profiles processed
55.78904303194852% done
104106 profiles processed
59.50831256741176% done
113574 profiles processed
63.22758210287499% done
123357 profiles processed
66.94685163833823% done
135387 profiles processed
70.66612117380147% done
144298 profiles processed
74.3853907092647% done
153007 profiles processed
78.104660244

In [5]:
temp_name = "temporaire.nc"
data_rep = configuration['CORA_OUTPUT']

In [6]:
Delete = True;
if(Delete):
    #os.remove(data_rep + configuration['OUTPUT_FILE_NAME'])
    #os.remove(data_rep + "temp_last.nc")
    os.remove(data_rep + temp_name)

In [7]:
ds = xr.open_dataset(data_rep +"ds_interp.nc")
ds = ds.dropna('N_PROF', how = "any", subset=['TEMP_INTERP', 'PSAL_INTERP'])
x = np.array(ds.TEMP_INTERP)

print(f"number of nan left: {np.count_nonzero(np.isnan(x))}")

#Select the period 
ds = ds.where(ds.JULD.dt.year<=configuration['YEARS_2']
              ,drop = True)
ds = ds.where(ds.JULD.dt.year>=configuration['YEARS_1'],drop = True)

ds_clean = ds.where(np.isnan(ds['TEMP_SURF'].isel(PRES_INTERPOLATED=14))==False, drop=True)
ds_clean = ds_clean.rename({'LAT': 'LATITUDE', 'LON':'LONGITUDE', 'JULD':'TIME'})
ds_clean.to_netcdf(data_rep + temp_name)

number of nan left: 0


In [8]:
%%time

#Load Data
data_rep = configuration['CORA_OUTPUT']
ds = xr.open_dataset(data_rep + temp_name)

lats = ds.LATITUDE
lons = ds.LONGITUDE
dates = ds.TIME

CPU times: user 12 ms, sys: 24 ms, total: 36 ms
Wall time: 41.1 ms


In [9]:
%%time
#BATHYMETRY
ds_bat = xr.open_dataset(configuration['BATHYMETRIE_output_path'] + configuration['BATHYMETRIE_FILE_NAME'])
res_lin = ds_bat.interp(LATITUDE=lats,LONGITUDE=lons,method = 'linear')['bathymetry'].values
ds = ds.assign(variables={"bathy": (('N_PROF'), res_lin)})

CPU times: user 128 ms, sys: 68 ms, total: 196 ms
Wall time: 209 ms


In [10]:
%%time
#MDT
ds_mdt = xr.open_dataset(configuration['MDT_DATA_path'])
ds_mdt = ds_mdt.assign_coords(lon180=(((ds_mdt.longitude + 180) % 360) - 180))  
ds_mdt['longitude'] = ds_mdt.lon180
res = ds_mdt.interp(latitude=lats,longitude=lons,method = 'nearest')['mdt'].values[0]
ds = ds.assign(variables={"MDT": (('N_PROF'), res)})


CPU times: user 604 ms, sys: 1.89 s, total: 2.49 s
Wall time: 2.71 s


In [11]:
%%time

#SST
path = configuration['SST_output_path']
ds_sst = xr.open_mfdataset(path + '*.nc')
spatial_domain = {"lon":[configuration['longitude1'], configuration['longitude2']],
                 "lat": [configuration['latitude2'], configuration['latitude1']]}

ds_sst = ds_sst.sel(lon=slice(spatial_domain['lon'][0], spatial_domain['lon'][1]),
                    lat=slice(spatial_domain['lat'][0], spatial_domain['lat'][1]))
res = ds_sst.sel(dict(lat=lats, lon=lons, time=dates), method='nearest')['analysed_sst'].values-273.15

ds = ds.assign(variables={"SST": (('N_PROF'), res)})

res = ds_sst.sel(dict(lat=lats, lon=lons, time=dates), method='nearest')['analysis_uncertainty'].values
ds = ds.assign(variables={"SST_uncertainty": (('N_PROF'), res)})


CPU times: user 26.3 s, sys: 18.6 s, total: 44.9 s
Wall time: 4min 10s


In [12]:
%%time

#SLA
path_SLA = configuration['SLA_path']
ds_sla = xr.open_mfdataset(path_SLA + '*.nc')
spatial_domain = {"lon":[configuration['longitude1'], configuration['longitude2']],
                 "lat": [configuration['latitude2'], configuration['latitude1']]}

ds_sla = ds_sla.sortby('longitude')

ds_sla = ds_sla.sel(longitude=slice(spatial_domain['lon'][0], spatial_domain['lon'][1]),
                    latitude=slice(spatial_domain['lat'][0], spatial_domain['lat'][1]))

res = ds_sla.sel(dict(latitude=lats, longitude=lons, time=dates), method='nearest')

ds = ds.assign(variables={"SLA": (('N_PROF'), res['sla'].values)}) 
ds = ds.assign(variables={"UGOS": (('N_PROF'), res['ugos'].values)}) 
ds = ds.assign(variables={"VGOS": (('N_PROF'), res['vgos'].values)}) 
ds = ds.assign(variables={"UGOSA": (('N_PROF'), res['ugosa'].values)}) 
ds = ds.assign(variables={"VGOSA": (('N_PROF'), res['vgosa'].values)}) 
ds = ds.assign(variables={"SLA_err": (('N_PROF'), res['err'].values)}) 


ds = ds.dropna('N_PROF', how = "all", subset=['SLA']) #Remove all NaN in SLA
ds = ds.dropna('N_PROF', how = "all", subset=['UGOS']) #Remove all NaN in SLA

ds.to_netcdf(data_rep + "temp_last.nc")


CPU times: user 34.1 s, sys: 3.82 s, total: 37.9 s
Wall time: 1min 26s


In [15]:
ds['LATITUDE'].attrs['standard_name'] = str(ds_sla['latitude'].standard_name)
ds['LATITUDE'].attrs['long_name'] = str(ds_sla['latitude'].long_name)
ds['LATITUDE'].attrs['axis'] = str(ds_sla['latitude'].axis)

ds['LONGITUDE'].attrs['standard_name'] = "longitde"
ds['LONGITUDE'].attrs['long_name'] = "longitde"
ds['LONGITUDE'].attrs['axis'] = "X"

ds['TIME'].attrs['axis'] = str(ds_sla['time'].axis)
ds['TIME'].attrs['standard_name'] = str(ds_sla['time'].standard_name)
ds['TIME'].attrs['long_name'] = str(ds_sla['time'].long_name)

ds['PRES_INTERPOLATED'].attrs['axis'] = "Z"
ds['PRES_INTERPOLATED'].attrs['standard_name'] = "interpolated pressure"
ds['PRES_INTERPOLATED'].attrs['long_name'] = "interpolated pressure"
ds['PRES_INTERPOLATED'].attrs['units'] = "Pa"

# -------------------

ds['TEMP_SURF'].attrs['standard_name'] = "surface temperature"
ds['TEMP_SURF'].attrs['long_name'] = "surface temperature"
ds['TEMP_SURF'].attrs['units'] = "Kelvin"

ds['PSAL_SURF'].attrs['standard_name'] = "surface salinity"
ds['PSAL_SURF'].attrs['long_name'] = "surface salinity"
ds['PSAL_SURF'].attrs['units'] = "Sa"

ds['TEMP_INTERP'].attrs['standard_name'] = "interpolated temperature"
ds['TEMP_INTERP'].attrs['long_name'] = "interpolated temperature"
ds['TEMP_INTERP'].attrs['units'] = "Kelvin"

ds['PSAL_INTERP'].attrs['standard_name'] = "interpolated salinity  "
ds['PSAL_INTERP'].attrs['long_name'] = "interpolated salinity  "
ds['PSAL_INTERP'].attrs['units'] = "Sa"


# -------------------
ds['bathy'].attrs['long_name'] = str(ds_bat['bathymetry'].long_name)

# -------------------
ds['MDT'].attrs['long_name'] = str(ds_mdt['mdt'].long_name)
ds['MDT'].attrs['units'] = str(ds_mdt['mdt'].units)
ds['MDT'].attrs['standard_name'] = str(ds_mdt['mdt'].standard_name)
ds['MDT'].attrs['grid_mapping'] = str(ds_mdt['mdt'].grid_mapping)

# -------------------
ds['SST'].attrs['units'] = str(ds_sst['analysed_sst'].units)
ds['SST_uncertainty'].attrs['units'] = str(ds_sst['analysis_uncertainty'].units)

# -------------------
ds['SST'].attrs['standard_name'] = str(ds_sst['analysed_sst'].standard_name)
ds['SST_uncertainty'].attrs['standard_name'] = str(ds_sst['analysis_uncertainty'].standard_name)

# -------------------
ds['SST'].attrs['long_name'] = str(ds_sst['analysed_sst'].long_name)
ds['SST_uncertainty'].attrs['long_name'] = str(ds_sst['analysis_uncertainty'].long_name)

# -------------------
ds['UGOS'].attrs['units'] = str(ds_sla['ugos'].units)
ds['VGOS'].attrs['units'] = str(ds_sla['vgos'].units)
ds['UGOSA'].attrs['units'] = str(ds_sla['ugosa'].units)
ds['VGOSA'].attrs['units'] = str(ds_sla['vgosa'].units)

ds['SLA'].attrs['units'] = str(ds_sla['sla'].units)
ds['SLA_err'].attrs['units'] = str(ds_sla['err'].units)

# ---------------------------------------------------------------- 

ds['UGOS'].attrs['standard_name'] = str(ds_sla['ugos'].standard_name)
ds['VGOS'].attrs['standard_name'] = str(ds_sla['vgos'].standard_name)
ds['UGOSA'].attrs['standard_name'] = str(ds_sla['ugosa'].standard_name)
ds['VGOSA'].attrs['standard_name'] = str(ds_sla['vgosa'].standard_name)

ds['SLA'].attrs['standard_name'] = str(ds_sla['sla'].standard_name)

# ---------------------------------------------------------------- 

ds['UGOS'].attrs['long_name'] = str(ds_sla['ugos'].long_name)
ds['VGOS'].attrs['long_name'] = str(ds_sla['vgos'].long_name)
ds['UGOSA'].attrs['long_name'] = str(ds_sla['ugosa'].long_name)
ds['VGOSA'].attrs['long_name'] = str(ds_sla['vgosa'].long_name)

ds['SLA'].attrs['long_name'] = str(ds_sla['sla'].long_name)
ds['SLA_err'].attrs['long_name'] = str(ds_sla['err'].long_name)

# ---------------------------------------------------------------- 

ds['UGOS'].attrs['grid_mapping'] = str(ds_sla['ugos'].grid_mapping)
ds['VGOS'].attrs['grid_mapping'] = str(ds_sla['vgos'].grid_mapping)
ds['UGOSA'].attrs['grid_mapping'] = str(ds_sla['ugosa'].grid_mapping)
ds['VGOSA'].attrs['grid_mapping'] = str(ds_sla['vgosa'].grid_mapping)

ds['SLA'].attrs['grid_mapping'] = str(ds_sla['sla'].grid_mapping)
ds['SLA_err'].attrs['grid_mapping'] = str(ds_sla['err'].grid_mapping)

In [16]:
ds

<xarray.Dataset>
Dimensions:            (N_PROF: 48142, PRES_INTERPOLATED: 51)
Coordinates:
    LONGITUDE          (N_PROF) float64 -14.33 -14.6 -13.5 ... -0.8517 -18.51
    LATITUDE           (N_PROF) float64 72.26 72.28 72.67 ... 60.59 70.98 68.56
  * PRES_INTERPOLATED  (PRES_INTERPOLATED) int64 0 1 2 3 4 ... 773 857 950 1000
    TIME               (N_PROF) datetime64[ns] 1993-05-25T01:24:00 ... 2019-0...
Dimensions without coordinates: N_PROF
Data variables: (12/16)
    TEMP_SURF          (N_PROF, PRES_INTERPOLATED) float64 ...
    PSAL_SURF          (N_PROF, PRES_INTERPOLATED) float64 ...
    TEMP_INTERP        (N_PROF, PRES_INTERPOLATED) float64 ...
    PSAL_INTERP        (N_PROF, PRES_INTERPOLATED) float64 ...
    DC_REFERENCE       (N_PROF) |S8 b'2643021' b'2643025' ... b'62951521'
    PLATFORM_NUMBER    (N_PROF) |S8 b'DESI' b'DESI' ... b'3901875' b'6902912'
    ...                 ...
    SLA                (N_PROF) float64 -0.0216 -0.0226 ... 0.1083 0.0356
    UGOS               (N_PROF) float64 0.0024 0.0011 -0.0067 ... 0.006 0.0062
    VGOS               (N_PROF) float64 -0.0188 -0.0235 ... -0.0043 -0.0531
    UGOSA              (N_PROF) float64 -0.0047 -0.0032 ... -0.0319 0.0191
    VGOSA              (N_PROF) float64 0.008 0.01 0.0088 ... -0.0099 0.0186
    SLA_err            (N_PROF) float64 0.0205 0.0208 0.0216 ... 0.0104 0.0082

In [17]:
%%time
    
#Find NaN if any
UGOS = np.array(ds.UGOS)
indna = np.argwhere(np.isnan(UGOS))[:,0]

if len(indna)>0:
    print(f"{len(indna)} prof to fix")
    
    print(indna) #print all N_PROF id of NaN
    
    fig = plt.figure(figsize=(12, 12))
    ax = plt.axes(projection=ccrs.PlateCarree())

    plt.title('Nan in SLA')

    ax.set_extent([17, -30, 50, 85], ccrs.PlateCarree())
    ax.coastlines(resolution='110m')

    histo_data = [];

    # add every Nan to map
    for i in range(len(indna)):
        lon_temp = np.array(ds.UGOS[indna[i]].LONGITUDE)
        lat_temp = np.array(ds.UGOS[indna[i]].LATITUDE)

        str = np.array2string(np.array(ds.UGOS[indna[i]].TIME))
        year = str[1:5]

        histo_data.append(year)

        plt.plot(lon_temp, lat_temp,  markersize=5, marker='o', color='red') #longitude * latide


    plt.show()

    fig = plt.figure(figsize=(12,12))

    plt.hist(histo_data)
    plt.title('NaN Evolution', fontsize=10)
    plt.show()
    
else:
    print("no prof to fix")

no prof to fix
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.01 ms


In [18]:
#Load Data
ds.to_netcdf(data_rep + "temp_last.nc")

dsub = ds.where((ds['LONGITUDE']>=configuration['longitude1']) & (ds['LONGITUDE']<=configuration['longitude2']) & (ds['LATITUDE']>=configuration['latitude2']) & 
              (ds['LATITUDE']<=configuration['latitude1']), drop=True)
ds = dsub #global or local

# reformating and renaming
s = ds.rename({"TEMP_INTERP": "TEMP", "PSAL_INTERP": "PSAL"}) # coder verification si le nom à déjà changer !

ds = s;

SA = gsw.SA_from_SP(ds['PSAL'], ds['PRES_INTERPOLATED'], ds['LONGITUDE'], ds['LATITUDE'])
CT = gsw.CT_from_t(SA, ds['TEMP'], ds['PRES_INTERPOLATED'])
SIG = gsw.sigma0(SA, CT)
ds = ds.assign(variables={"SA": (('N_PROF', 'PRES_INTERPOLATED'), SA.data)})
ds = ds.assign(variables={"CT": (('N_PROF', 'PRES_INTERPOLATED'), CT.data)})
ds = ds.assign(variables={"SIG": (('N_PROF', 'PRES_INTERPOLATED'), SIG.data)})

sig_diff = ds.SIG - ds.SIG.sel(PRES_INTERPOLATED = 10)-0.03
MLD_obs = ds['PRES_INTERPOLATED'].where(sig_diff>0).min(dim='PRES_INTERPOLATED')
ds = ds.assign(MLD = MLD_obs)

print('deleting temporary file ..')

os.remove(data_rep + "temp_last.nc")
os.remove(data_rep + temp_name)

print("removed")
print("Starting generation finalDataset File")
ds.to_netcdf(data_rep + configuration['OUTPUT_FILE_NAME'])
print("done !")

deleting temporary file ..
removed
Starting generation finalDataset File
done !
